In [27]:
!pip install xgboost

In [27]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('./train/*.csv'))]

In [28]:
know_train[0].head()

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,...,52,2,실업,1,1,1,4000,,2200,825101
1,1,4,5,4,5,3,4,3,4,3,...,38,4,건축공학,1,1,1,,,2400,140204
2,2,3,4,3,4,3,4,5,6,4,...,50,4,건축공학,1,1,1,4000,,2400,140204
3,3,3,3,3,3,3,5,4,5,4,...,42,4,환경학과,1,1,1,7000,,3500,140601
4,4,4,5,3,4,3,4,4,5,3,...,51,4,건축공학,1,1,1,4000,,2500,140204


In [29]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [30]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


In [31]:
train_data = {}
for year, df in zip(years, know_train):
    train_data[year] = {'X': df.iloc[:, 1:-1], # ID제외
                        'y': df.iloc[:, -1]} 

In [32]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [33]:
xgb_models = {}
params= {
#         'learning_rate': [0.03, 0.01, 0.003, 0.001],
        'min_child_weight': [1,3, 5,7, 10],
        'gamma': [0, 0.5, 1, 1.5, 2, 2.5, 5],
#         'subsample': [0.6, 0.8, 1.0, 1.2, 1.4],
        'colsample_bytree': [0.6, 0.8, 1.0, 1.2, 1.4],
        'max_depth': [3, 4, 5, 6, 7, 8, 9 ,10, 12, 14]
#         'reg_lambda':np.array([0.4, 0.6, 0.8, 1, 1.2, 1.4])
}

In [21]:
# model = XGBClassifier()

In [22]:
# rs = RandomizedSearchCV(model, param_distributions=params,n_jobs=-1, random_state=42, cv=3 )
#

In [25]:
# rs.fit(train_data['2017']['X'].iloc[:, :], train_data['2017']['y'])

[01:43:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints...
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None,
                                    

In [26]:
rs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6,
              enable_categorical=False, gamma=1, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
for year in tqdm(years):
    model = XGBClassifier()
    rs = RandomizedSearchCV(model, param_distributions=params,
                                    n_jobs=-1, random_state=42, cv=3 )
    rs.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    f_rs = rs.best_estimator_
    xgb_models[year] = f_rs

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

[02:22:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 25%|████████████████████                                                            | 1/4 [31:45<1:35:17, 1905.77s/it]

[02:51:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 50%|███████████████████████████████████████                                       | 2/4 [1:00:43<1:01:50, 1855.26s/it]

[03:19:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 75%|████████████████████████████████████████████████████████████                    | 3/4 [1:28:46<30:03, 1803.55s/it]

[03:49:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [1:58:50<00:00, 1782.58s/it]


In [41]:
xgb_models

{'2017': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.6,
               enable_categorical=False, gamma=1, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               learning_rate=0.300000012, max_delta_step=0, max_depth=7,
               min_child_weight=3, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=8, num_parallel_tree=1,
               objective='multi:softprob', predictor='auto', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
               tree_method='exact', validate_parameters=1, verbosity=None),
 '2018': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.6,
               enable_categorical=False, gamma=1, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               learning_rate=0.3000

In [42]:
params ={
    'n_estimators':[100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]}


In [43]:
from sklearn.ensemble import RandomForestClassifier
rf_models = {}

for year in tqdm(years):
    model = RandomForestClassifier(random_state = 42,n_jobs=-1)
    rrs = RandomizedSearchCV(model, param_distributions=params,
                                    n_jobs=-1, random_state=42, cv=3 )
    rrs.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    f_rrs = rrs.best_estimator_
    rf_models[year] = f_rrs

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:46<00:00, 26.58s/it]


In [44]:
know_test = [pd.read_csv(path) for path in sorted(glob('./test/*.csv'))]
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,,3,...,2,26,3,비서학,1,1,1,3000,,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,농화학,1,1,1,5500,,2500
2,2,5,5,5,4,5,4,1,,1,...,1,31,4,신문방송,1,1,1,4300,,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,화학,1,1,1,4100,,3000
4,4,5,6,4,5,4,5,1,,1,...,1,36,4,광고홍보,1,1,1,2800,,2000


In [45]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [46]:
years = ['2017', '2018', '2019', '2020']

for year, df in zip(years, know_test):
    print(year)
    encoders = {}
    
    for col in df.columns:
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = year_encoder[year][col]
            df[col] = df[col].map(str)
            category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
            df[col] = df[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
            

2017
2018
2019
2020


In [47]:
know_train[0]['aq1_1'].value_counts()

3    3116
4    3007
2    1706
5    1072
1     585
Name: aq1_1, dtype: int64

In [48]:
test_data = {}
for year, df in zip(years, know_test):
    test_data[year] =  {'X': df.iloc[:,1:]}

In [49]:
xgb_predicts = [] 

for year in tqdm(years):
    pred = xgb_models[year].predict(test_data[year]['X'])
    xgb_predicts.extend(pred)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.88it/s]


In [50]:
rf_predicts = [] 

for year in tqdm(years):
    pred = rf_models[year].predict(test_data[year]['X'])
    rf_predicts.extend(pred)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.59s/it]


In [51]:
submission = pd.read_csv('sample_submission.csv') # sample submission 불러오기

In [52]:
submission['knowcode'] = xgb_predicts

submission.to_csv('submission_xgb.csv', index=False)

In [53]:
submission['knowcode'] = rf_predicts

submission.to_csv('submission_rf.csv', index=False)